In [1]:
import statistics as st

In [2]:
import argparse
import logging
import os

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

from unet import UNet
from utils.data_vis import plot_img_and_mask
from utils.dataset_old import BasicDataset
import cv2

In [3]:
def predict_img(net,
                full_img,
                device,
                scale_factor=1,
                out_threshold=0.5):
    net.eval()

    img = torch.from_numpy(BasicDataset.preprocess(full_img, scale_factor))

    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        output = net(img)

        probs_0 = torch.sigmoid(output[:, 0, :, :])
        probs_1 = torch.sigmoid(output[:, 1, :, :])

        probs_0 = probs_0.squeeze(0)
        probs_1 = probs_1.squeeze(0)

        tf = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.Resize(full_img.width), #size[1]),
                transforms.ToTensor()
            ]
        )

        probs_0 = tf(probs_0.cpu())
        probs_1 = tf(probs_1.cpu())
        mask_0 = probs_0.squeeze().cpu().numpy()
        mask_1 = probs_1.squeeze().cpu().numpy()
        full_mask = np.array([mask_0, mask_1]) #probs.squeeze().cpu().numpy()

    return full_mask# > out_threshold


def get_output_filenames(args):
    in_files = args.input
    out_files = []

    if not args.output:
        for f in in_files:
            pathsplit = os.path.splitext(f)
            out_files.append("{}_OUT{}".format(pathsplit[0], pathsplit[1]))
    elif len(in_files) != len(args.output):
        logging.error("Input files and output files are not of the same length")
        raise SystemExit()
    else:
        out_files = output

    return out_files


def mask_to_image(mask):
    return Image.fromarray((mask * 255).astype(np.uint8))

In [4]:
model_name = 'sag_unet'
model = 'ckpts_dir/sag_ckpts/best_epoch_147.pth'

net = UNet(n_channels=1, n_classes=2)

logging.info("Loading model {}".format(model))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')
net.to(device=device)
net.load_state_dict(torch.load(model, map_location=device))

logging.info("Model loaded !")

scale = 0.5
mask_threshold = 0.5 #минимальная вероятность для рассматривания пикселя на маску

no_save = True
viz = False

list_of_dirs = [
    '/home/vlad/dataset_new/test_data/'
]

In [5]:
img_list = os.listdir(list_of_dirs[0]+'imgs/') 
masks_list = os.listdir(list_of_dirs[0]+'masks/') 

img_src_pathes = []
for img in img_list:
    img_src_pathes.append(list_of_dirs[0]+'imgs/'+img)
    
mask_src_pathes = []
for mask in masks_list:
    mask_src_pathes.append(list_of_dirs[0]+'masks/'+mask)

In [8]:
iou_general = []
f1_general = []

bad_iou = []
bad_results = []

for i in range(0, len(img_src_pathes)):
    imgname = img_src_pathes[i]
    maskname = mask_src_pathes[i]
    
    img = Image.open(imgname)
    true_mask = cv2.imread(maskname)

    mask = predict_img(net=net,
                       full_img=img,
                       scale_factor=scale,
                       out_threshold=mask_threshold,
                       device=device)

    res_treshold = 0.75 #.75 best for 147th epoch
    mask = np.array([mask[0] > res_treshold, mask[1] > res_treshold])
    
    d, w, h = mask.shape
    
    true_mask2 = cv2.resize(true_mask[:,:,0], (h, w))
    
    print(f'true mask shape = {true_mask.shape}')
    print(f'pred mask shape = {mask.shape}')
    print(f'true mask 2 shape = {true_mask2.shape}')
    break

    pixelThreshold = 0.5
    bin_mask = np.where(mask[0] > pixelThreshold, 1, 0)

    unique, counts = np.unique(bin_mask + true_mask2, return_counts=True)
    inters = counts[-1]
    union = counts[-1]+counts[-2]

    iou = inters/union

    iou_general.append(iou)

    if iou<0.5:
        bad_iou.append([iou, imgname])

#     squares = 2*h*w
    un, cnts = np.unique(bin_mask, return_counts=True)
    un, cnts2 = np.unique(true_mask2, return_counts=True)
    squares = cnts[1] + cnts2[1]

    f1 = 2 * inters / squares
    f1_general.append(f1)
    
iou_max = max(iou_general)
iou_max_img = np.argmax(iou_general)
iou_min = min(iou_general)
iou_min_img = np.argmin(iou_general)

f1_max = max(f1_general)
f1_max_img = np.argmax(f1_general)
f1_min = min(f1_general)
f1_min_img = np.argmin(f1_general)
    
print(f'f1_score max = {f1_max}, image = {img_src_pathes[f1_max_img]}')
print(f'f1_score min = {f1_min}, image = {img_src_pathes[f1_min_img]}')
print(f'iou max = {iou_max}, image = {img_src_pathes[iou_max_img]}')
print(f'iou min = {iou_min}, image = {img_src_pathes[iou_min_img]}') 
print(min(np.min(iou_general[:iou_min_img]), np.min(iou_general[iou_min_img + 1:])))
    

true mask shape = (480, 512, 3)
pred mask shape = (2, 512, 546)
true mask 2 shape = (512, 546)


ValueError: max() arg is an empty sequence

In [ ]:
print(np.argmax(iou_general))
print(img_src_pathes[np.argmax(iou_general)])
print()

In [ ]:
with open('log_score/log_'+model_name+'_'+model.split('/')[-1]+'_' + str(res_treshold) + '.txt', 'w') as file:
    file.write('General metrics: \n')
    file.write('------------------------\n')
    file.write('Max IOU:\n')
    file.write(str(max(iou_general))+'\n')
    file.write(' ---\n')
    
    file.write('Max f1:\n')
    file.write(str(max(f1_general))+'\n')
    file.write(' ---\n')
    
    file.write('Min IOU:\n')
    file.write(str(min(iou_general))+'\n')
    file.write(' ---\n')
    
    file.write('Min f1:\n')
    file.write(str(min(f1_general))+'\n')
    file.write(' ---\n')
    
    file.write('Mean IOU:\n')
    file.write(str(st.mean(iou_general))+'\n')
    file.write(' ---\n')
    
    file.write('Mean f1:\n')
    file.write(str(st.mean(f1_general))+'\n')
    file.write(' ---\n')
    
    file.write('\n')
    file.write('\n')
    
    file.write('-------------------\n')
    file.write('Bad iou:\n')
    for el in bad_iou:
        file.write(str(el[0])+'  __  '+str(el[1]) + '\n')
        
    file.write('\n')
    file.write('\n')
    
    file.write('-------------------\n')
    file.write('Bad calculations:\n')
    for el in bad_results:
        file.write(str(el[0])+'  __  '+str(el[1]) +'  __  ' + str(el[2]) + '\n')

CP_epoch100.pth  CP_epoch150.pth  CP_epoch200.pth  CP_epoch250.pth  CP_epoch300.pth  CP_epoch350.pth  CP_epoch400.pth  CP_epoch50.pth

In [ ]:
model_name = 'trained_by_vlad'

# weights_lst = ['CP_epoch50.pth',
#                'CP_epoch100.pth',

# weights_lst = ['CP_epoch150.pth',
#                'CP_epoch200.pth', 
#                'CP_epoch250.pth', 
#                'CP_epoch300.pth', 
#                'CP_epoch350.pth', 
#                'CP_epoch400.pth']

weights_lst = ['CP_epoch400.pth']

res_threshold_lst = np.linspace(0, 1, 9)
print(res_threshold_lst)

net = UNet(n_channels=1, n_classes=2)

for weights_path in weights_lst:
    weights = os.path.join('/home/vlad/unet4/ckpts_dir_new_0/mid_ckpts/', weights_path)
    print(weights)
    logging.info("Loading model {}".format(weights))

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f'Using device {device}')
    net.to(device=device)
    net.load_state_dict(torch.load(weights, map_location=device))

    logging.info("Model loaded!")

    scale = 0.5
    mask_threshold = 0.5 #минимальная вероятность для рассматривания пикселя на маску

    no_save = True
    viz = False

    list_of_dirs = [
        '/home/vlad/dataset_new/test_data/'
    ]

    img_list = os.listdir(list_of_dirs[0]+'imgs/') 
    masks_list = os.listdir(list_of_dirs[0]+'masks/') 

    img_src_pathes = []
    for img in img_list:
        img_src_pathes.append(list_of_dirs[0]+'imgs/'+img)

    mask_src_pathes = []
    
    for mask in masks_list:
        mask_src_pathes.append(list_of_dirs[0]+'masks/'+mask)
        
    for res_treshold in res_threshold_lst:
        iou_general = []
        f1_general = []

        bad_iou = []
        bad_results = []

        for i in range(0, len(img_src_pathes)):
            imgname = img_src_pathes[i]
            maskname = mask_src_pathes[i]

            img = Image.open(imgname)
            true_mask = cv2.imread(maskname)

            mask = predict_img(net=net,
                               full_img=img,
                               scale_factor=scale,
                               out_threshold=mask_threshold,
                               device=device)
            ########
            ### change the threshold param for better / worse results

#             res_treshold = 0.05 #.75 best for 147th epoch
            mask = np.array([mask[0] > res_treshold, mask[1] > res_treshold])

            ########


            d, w, h = mask.shape

            try:

                true_mask2 = cv2.resize(true_mask[:,:,0], (h, w))

                pixelThreshold = 0.5
                bin_mask = np.where(mask[0] > pixelThreshold, 1, 0)

                unique, counts = np.unique(bin_mask + true_mask2, return_counts=True)
                inters = counts[-1]
                union = counts[-1]+counts[-2]

                iou = inters/union

                iou_general.append(iou)

                if iou<0.5:
                    bad_iou.append([iou, imgname])

            #     squares = 2*h*w
                un, cnts = np.unique(bin_mask, return_counts=True)
                un, cnts2 = np.unique(true_mask2, return_counts=True)
                squares = cnts[1] + cnts2[1]

                f1 = 2 * inters / squares
                f1_general.append(f1)
                
            except:
                print('Упс, ошибочка вышла(')
                bad_results.append([imgname, maskname, i])
                
                
        with open('log_score/log_'+model_name+'_'+model.split('/')[-1]+'_' + str(res_treshold) + '.txt', 'w') as file:
            file.write('General metrics: \n')
            file.write('------------------------\n')
            file.write('Max IOU:\n')
            file.write(str(max(iou_general))+'\n')
            file.write(' ---\n')

#             file.write('Max f1:\n')
#             file.write(str(max(f1_general))+'\n')
#             file.write(' ---\n')

            file.write('Min IOU:\n')
            file.write(str(min(iou_general))+'\n')
            file.write(' ---\n')

#             file.write('Min f1:\n')
#             file.write(str(min(f1_general))+'\n')
#             file.write(' ---\n')

            file.write('Mean IOU:\n')
            file.write(str(st.mean(iou_general))+'\n')
            file.write(' ---\n')

#             file.write('Mean f1:\n')
#             file.write(str(st.mean(f1_general))+'\n')
#             file.write(' ---\n')

            file.write('\n')
            file.write('\n')

            file.write('-------------------\n')
            file.write('Bad iou:\n')
            for el in bad_iou:
                file.write(str(el[0])+'  __  '+str(el[1]) + '\n')

            file.write('\n')
            file.write('\n')

            file.write('-------------------\n')
            file.write('Bad calculations:\n')
            for el in bad_results:
                file.write(str(el[0])+'  __  '+str(el[1]) +'  __  ' + str(el[2]) + '\n')

In [ ]:
iou_general = []
f1_general = []

bad_iou = []
bad_results = []

for i in range(0, len(img_src_pathes)):
    imgname = img_src_pathes[i]
    maskname = mask_src_pathes[i]
    
    img = Image.open(imgname)
    true_mask = cv2.imread(maskname)

    mask = predict_img(net=net,
                       full_img=img,
                       scale_factor=scale,
                       out_threshold=mask_threshold,
                       device=device)
    ########
    ### change the threshold param for better / worse results

    res_treshold = 0.05 #.75 best for 147th epoch
    mask = np.array([mask[0] > res_treshold, mask[1] > res_treshold])

    ########

    
    d, w, h = mask.shape
    
#     try:
    
    true_mask2 = cv2.resize(true_mask[:,:,0], (h, w))

    pixelThreshold = 0.5
    bin_mask = np.where(mask[0] > pixelThreshold, 1, 0)

    unique, counts = np.unique(bin_mask + true_mask2, return_counts=True)
    inters = counts[-1]
    union = counts[-1]+counts[-2]

    iou = inters/union

    iou_general.append(iou)

    if iou<0.5:
        bad_iou.append([iou, imgname])

#     squares = 2*h*w
    un, cnts = np.unique(bin_mask, return_counts=True)
    un, cnts2 = np.unique(true_mask2, return_counts=True)
    squares = cnts[1] + cnts2[1]

    f1 = 2 * inters / squares
    f1_general.append(f1)

#     except:
#     print('Упс, ошибочка вышла(')
#     bad_results.append([imgname, maskname, i])

In [ ]:
with open('log_score/log_'+model_name+'_'+model.split('/')[-1]+'_' + str(res_treshold) + '.txt', 'w') as file:
    file.write('General metrics: \n')
    file.write('------------------------\n')
    file.write('Max IOU:\n')
    file.write(str(max(iou_general))+'\n')
    file.write(' ---\n')
    
    file.write('Max f1:\n')
    file.write(str(max(f1_general))+'\n')
    file.write(' ---\n')
    
    file.write('Min IOU:\n')
    file.write(str(min(iou_general))+'\n')
    file.write(' ---\n')
    
    file.write('Min f1:\n')
    file.write(str(min(f1_general))+'\n')
    file.write(' ---\n')
    
    file.write('Mean IOU:\n')
    file.write(str(st.mean(iou_general))+'\n')
    file.write(' ---\n')
    
    file.write('Mean f1:\n')
    file.write(str(st.mean(f1_general))+'\n')
    file.write(' ---\n')
    
    file.write('\n')
    file.write('\n')
    
    file.write('-------------------\n')
    file.write('Bad iou:\n')
    for el in bad_iou:
        file.write(str(el[0])+'  __  '+str(el[1]) + '\n')
        
    file.write('\n')
    file.write('\n')
    
    file.write('-------------------\n')
    file.write('Bad calculations:\n')
    for el in bad_results:
        file.write(str(el[0])+'  __  '+str(el[1]) +'  __  ' + str(el[2]) + '\n')